# Databases and SQL

Chapter 15 of Py4E gets into databases and the first sections are a good place to start.

## What is a database?
I do like Charles Severance's (Py4E author) idea that, at its foundation
>"a database is a file that is organized to store data" (Section 15.1 of Py4E). 



Charles goes on to liken databases to Python dictionaries, except that they **store their data on disk** rather than in a data structure in memory. This allows a database to store more data.

Databases are designed not only to store data, but to access that data--and access it quickly! Indexing the stored data is one method used by database systems to speed up data access.


## Database Systems
### Also called Relational Database Managements Systems (RDMS)
 * Usually server-based
 * Client-Server model
 * Performance and Scalability
 * ACID (Atomicity, Consistency, Isolation, Durability)
 

![ACID Image](https://s3-us-west-1.amazonaws.com/morpheus-staging/system/spud_media/332/original/acidoverwiew.png?1422558433)



Databases can range from the example we'll use below, which requires nothing more than the Python install we've been using all along, to supercomputer sized systems used by large corporations. More than just a file, databases are managed by their own software, and often that software runs on a server.

While, we'll use SQLite for this class, which is designed to be light-weight and can be embedded in other applications (like your web browser as Py4E points out), the same concepts scale up to larger databases and server-based systems.

Why use a server? For one, as databases grow, their files grow. Also more compute power is needed to quickly access that data. So, for larger databases, larger computers, typically servers are needed--*this can also scale to clusters of computers*. Also, larger databases tend to be used by more than one person at a time. Server-based databases are typically setup with a **client-server model**, where the database is running on a server that users to not actively log into. Users connect to the server using the client. This allows the server to be dedicated to processing client queries and allows for the server to be setup to manage concurrent use from multiple users. If multiple users try to change the same file on a computer at the same time, file corruption can happen. Using the client-server model prevents this.  



### Common Database Systems

 * [SQLite](https://sqlite.org/index.html)
     * Free
     * Small
     * Not server-based
 * [MySQL](https://www.mysql.com/)
     * Free (Community Server)
     * Relatively common and robust
 * [PostgreSQL](https://www.postgresql.org/)
     * Free
     * Geospatial extensions
 * [Oracle](https://www.oracle.com/index.html)
     * Commercial
     * UF Licensed
 
         

 ### Desktop databases software
 * FileMaker
 * Microsoft Access



### NOSQL (Non SQL, or Not Only SQL) Databases
 * [MongoDB](https://www.mongodb.com/)
    * A document-based DB
    * MongoDB stores data in flexible, JSON-like documents, meaning fields can vary from document to document and data structure can be changed over time
 * [Cassandra](http://cassandra.apache.org/)
     * A wide column DB
 * [neo4j](https://neo4j.com/)
     * A graph-based DB
     ![neo4j graph representation](https://s3.amazonaws.com/dev.assets.neo4j.com/wp-content/uploads/20170731095054/Property-Graph-Concepts-Simple.svg)


## Tables

### Revisiting [Browman & Woo (2018)](https://www.tandfonline.com/doi/full/10.1080/00031305.2017.1375989)

![Figure 2 of Browman & Woo 2018, showinf examples of spreadsheets that violate best practices](https://www.tandfonline.com/na101/home/literatum/publisher/tandf/journals/content/utas20/2018/utas20.v072.i01/00031305.2017.1375989/20180424-01/images/large/utas_a_1375989_f0002_b.jpeg)

We've worked with tables a lot already in class. Every problem set has had a table of data, we've parsed tables reading line-by-line, we've loaded tables into Pandas dataframes, and we had a reading ([Browman and Woo 2018)](https://www.tandfonline.com/doi/full/10.1080/00031305.2017.1375989)) that made recommendations for how to best store data in tables (spreadsheets are essentially tables of data).

Databases extend tables.

# Our First Database!

Python has SQLite built in, you just need to import it.



In [12]:
import sqlite3

conn = sqlite3.connect('music.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Tracks')
cur.execute('CREATE TABLE Tracks (title TEXT, plays INTEGER)') 

conn.close()

# Code: http://www.py4e.com/code3/db1.py


## A lot going on here...

```python
conn = sqlite3.connect('music.sqlite')
cur = conn.cursor()
```
 * Makes a connection, called `conn` in this case, to our database
 * Connection, because databases are often on a server
     * Note that we now have a file called music.sqlite in the current folder
 * Call the `cursor()` method on the connection--similar to `open()` on a file
 * `cur` is similar to a file handle or web socket

Py4E Figure 15.2: A database cursor
![Py4E Figure 15.2: A database cursor](https://www.py4e.com/images/cursor.svg)

```python
cur.execute('DROP TABLE IF EXISTS Tracks')
cur.execute('CREATE TABLE Tracks (title TEXT, plays INTEGER)') 
```

 * Use the `cur` cursor to execute commands on the connected DB
 * Drop (delete) the table "Tracks" if is exists
 * Create a table "Tracks" with two columns:
     * "title", a text column
     * "plays", an integer column


# Data types

 * Databases typically are very strict about types
 * SQLite is an exception:
 >SQLite is "typeless". This means that you can store any kind of data you want in any column of any table, regardless of the declared datatype of that column...This behavior is a feature, not a bug. ... The strong typing system found in most other SQL engines and codified in the SQL language spec is a misfeature. [SQLite Datatypes Page](https://www.sqlite.org/datatypes.html)

## Some MySQL data types
|Data Type| Description
|-------|---------------|
|INTEGER|An integer|
|FLOAT| Real numbers, including scientific notation|
|DATE|YYYY-MM-DD|
|DATETIME|YYYY-MM-DD HH:MM:SS|
|TEXT| Strings up to 65535 characters|
|TINYTEXT|Strings up to 255 characters|
|BLOB|Binary Large Object (e.g., images)|

## Let's add data to our music database

In [13]:
import sqlite3

conn = sqlite3.connect('music.sqlite')
cur = conn.cursor()

cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)',
    ('Thunderstruck', 20))
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)',
    ('My Way', 15))
conn.commit()

print('Tracks:')
cur.execute('SELECT title, plays FROM Tracks') 
for row in cur:
    print(row)

cur.execute('DELETE FROM Tracks WHERE plays < 100') 
    
cur.close()
# Code: http://www.py4e.com/code3/db2.py

Tracks:
('Thunderstruck', 20)
('My Way', 15)


## This looks complicated

```python
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)',
    ('Thunderstruck', 20))
```



```python
conn.commit()
```
From [StackOverflow](https://stackoverflow.com/questions/2847999/why-the-need-to-commit-explicitly-when-doing-an-update)
> The DB-API spec requires that connecting to the database begins a new transaction, by default. You must `commit` to confirm any changes you make, or `rollback` to discard them.



```python
cur.execute('SELECT title, plays FROM Tracks') 
for row in cur:
    print(row)
```

> ('Thunderstruck', 20)

>  ('My Way', 15)

```python
cur.execute('DELETE FROM Tracks WHERE plays < 100') 
```

# SQL
 * Structured Query Language
     * Upper case by convention, but not needed
     * We will cover some basics
         * But we will largely use a Python module that converts Python methods to SQL for us
     